# Import libraries & Import dataset

In [90]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree

#Loading dataset
df = pd.read_csv('Immo_Sale_24_01_2023_00_03_03.csv')
df.head(1)


,id,price,living_area,property_type,rooms_number,zip_code,land_area,garden,garden_area,equipped_kitchen,swimming_pool,furnished,open_fire,terrace,terrace_area,facades_number,building_state,province,region,building_condition
0,10147032,159000,122,house,2,4430,335,1,171,1,0,0,0,1,20,2,0,Liège,Wallonia,Good


# Understand the data

In [91]:
# Get the no of rows and columns
df.shape

(13610, 20)

In [92]:
#Get all the column names
df.columns

Index(['id', 'price', 'living_area', 'property_type', 'rooms_number',
       'zip_code', 'land_area', 'garden', 'garden_area', 'equipped_kitchen',
       'swimming_pool', 'furnished', 'open_fire', 'terrace', 'terrace_area',
       'facades_number', 'building_state', 'province', 'region',
       'building_condition'],
      dtype='object')

# Step 1: Data cleaning

Handling duplicates

In [93]:

    #Remove duplicates 
df.drop_duplicates(inplace=True)
    #No blank spaces. T
df.dropna(how='all') 



,id,price,living_area,property_type,rooms_number,zip_code,land_area,garden,garden_area,equipped_kitchen,swimming_pool,furnished,open_fire,terrace,terrace_area,facades_number,building_state,province,region,building_condition
0,10147032,159000,122,house,2,4430,335,1,171,1,0,0,0,1,20,2,0,Liège,Wallonia,Good
1,10146903,345000,119,house,4,9000,49,0,0,1,0,0,0,0,0,2,0,Oost-Vlaanderen,Flanders,Good
2,10146862,895000,200,house,3,1150,300,1,200,1,0,0,0,1,12,3,0,Vlaams-Brabant,Brussels,To renovate
3,9935270,570000,404,house,4,1570,1000,1,1000,1,0,0,0,1,52,4,1,Vlaams-Brabant,Flanders,As new
4,10143254,425000,130,house,3,4950,991,0,0,1,0,0,0,1,14,4,0,Liège,Wallonia,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13605,9569187,329000,126,apartment,2,9200,126,0,0,0,0,0,0,0,0,1,1,Oost-Vlaanderen,Flanders,As new
13606,9947895,329500,84,apartment,2,8370,84,0,0,1,0,0,0,1,6,2,0,West-Vlaanderen,Flanders,To renovate
13607,9947853,329500,80,apartment,2,8370,80,0,0,1,0,0,0,1,18,2,0,West-Vlaanderen,Flanders,To renovate
13608,9947891,329750,90,apartment,2,8370,90,0,0,1,0,0,0,1,9,2,0,West-Vlaanderen,Flanders,To renovate


Handling null values

In [94]:

# Get the sum of all na values from dataset
df.isna().sum()

id                    0
price                 0
living_area           0
property_type         0
rooms_number          0
zip_code              0
land_area             0
garden                0
garden_area           0
equipped_kitchen      0
swimming_pool         0
furnished             0
open_fire             0
terrace               0
terrace_area          0
facades_number        0
building_state        0
province              0
region                0
building_condition    0
dtype: int64

Since all my training examples not containing null values, lets check the shape of the dataset again

In [95]:
df.shape

(13610, 20)

creating new column building_cond on the basis of building_condition

In [75]:
df.building_condition.value_counts()

Good              4073
As new            3923
To be done up     1264
To renovate       1121
Just renovated     714
To restore          44
Name: building_condition, dtype: int64

In [76]:
#To evaluate i am only taking 'As New','Good' and 'Just renovated' and eleminating 'to renovate', 'To be done up'
# and 'To restore' for my model

def building_cond(building_condition):

    if building_condition == 'As new':
        return "1"
   
    elif building_condition == 'Good':
        return "2"
    
    elif building_condition == 'Just renovated':
        return "3"
    else: 
        return "0"
df.insert(2, 'building_cond', df.building_condition.apply(building_cond))

created new column region_no on the basis of zipcode

In [77]:
df.province.value_counts()

Vlaams-Brabant     3731
West-Vlaanderen    2331
Oost-Vlaanderen    2027
Liège              1263
Antwerpen          1234
Hainaut            1054
Brabant Wallon      871
Luxembourg          464
Namur               410
Limburg             225
Name: province, dtype: int64

In [78]:
def region_no(zip_code):
   #Brussels=1
    if 1000 <= zip_code < 1300:
        return "1"
    #Wallon Brabant: 2
    elif 1300 <= zip_code < 1500:
        return "2"
     #Flemish Brabant: 3
    elif (zip_code >= 1500 and zip_code < 2000) or (zip_code >= 3000 and zip_code < 3500):
        return "3"
    #Antwerp: 4
    elif 2000 <= zip_code < 3000:
        return "4"
     #Limburg: 5
    elif 3500 <= zip_code < 4000:
        return "5"
    #Liège: 6
    elif 4000 <= zip_code < 5000:
        return "6"
    #Namur: 7
    elif 5000 <= zip_code < 6000:
        return "7"
    #Hainaut: 8
    elif (zip_code >= 6000 and zip_code < 6600) or (zip_code >= 7000 and zip_code < 8000):
        return "8" 
    #Luxembourg: 9
    elif 6600 <= zip_code < 7000:
        return "9"
    #West Flanders: 10
    elif 8000 <= zip_code < 9000:
        return "10" 
    #East Flanders: 11 
    else: 
        return "11"
df.insert(1, 'region_no', df.zip_code.apply(region_no))

In [79]:
df.head()

,id,region_no,price,building_cond,living_area,property_type,rooms_number,zip_code,land_area,garden,...,swimming_pool,furnished,open_fire,terrace,terrace_area,facades_number,building_state,province,region,building_condition
0,10147032,6,159000,2,122,house,2,4430,335,1,...,0,0,0,1,20,2,0,Liège,Wallonia,Good
1,10146903,11,345000,2,119,house,4,9000,49,0,...,0,0,0,0,0,2,0,Oost-Vlaanderen,Flanders,Good
2,10146862,1,895000,0,200,house,3,1150,300,1,...,0,0,0,1,12,3,0,Vlaams-Brabant,Brussels,To renovate
3,9935270,3,570000,1,404,house,4,1570,1000,1,...,0,0,0,1,52,4,1,Vlaams-Brabant,Flanders,As new
4,10143254,6,425000,2,130,house,3,4950,991,0,...,0,0,0,1,14,4,0,Liège,Wallonia,Good


In [80]:
#droped the columns not required for my analysis
df.drop(columns =['id', 'province', 'region', 'province', 'region','building_condition'], axis=1)

,region_no,price,building_cond,living_area,property_type,rooms_number,zip_code,land_area,garden,garden_area,equipped_kitchen,swimming_pool,furnished,open_fire,terrace,terrace_area,facades_number,building_state
0,6,159000,2,122,house,2,4430,335,1,171,1,0,0,0,1,20,2,0
1,11,345000,2,119,house,4,9000,49,0,0,1,0,0,0,0,0,2,0
2,1,895000,0,200,house,3,1150,300,1,200,1,0,0,0,1,12,3,0
3,3,570000,1,404,house,4,1570,1000,1,1000,1,0,0,0,1,52,4,1
4,6,425000,2,130,house,3,4950,991,0,0,1,0,0,0,1,14,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13605,11,329000,1,126,apartment,2,9200,126,0,0,0,0,0,0,0,0,1,1
13606,10,329500,0,84,apartment,2,8370,84,0,0,1,0,0,0,1,6,2,0
13607,10,329500,0,80,apartment,2,8370,80,0,0,1,0,0,0,1,18,2,0
13608,10,329750,0,90,apartment,2,8370,90,0,0,1,0,0,0,1,9,2,0


In [81]:
houses = df.drop(df[(df['property_type'] != 'house')].index)
houses.to_csv('houses1.csv')

In [82]:
houses = df.drop(df[(df['property_type'] != 'apartment')].index)
houses.to_csv('apartment.csv')